Countries Analysis

In [73]:
import pandas as pd
import requests
import json


In [74]:
# Fecthing the dat from api
api_url = 'https://restcountries.com/v3.1/all'
response = requests.get(api_url)
data = response.json()

In [75]:
# Converting it into a datframe
# df = pd.DataFrame(data)
# print(df.info())
df = pd.json_normalize(data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Columns: 897 entries, tld to languages.lit
dtypes: bool(3), float64(20), int64(1), object(873)
memory usage: 1.7+ MB


In [79]:
df_cols_to_drop = [col for col in df.columns if 'translations' in col or 'currencies' in col or 'coatOfArms' in col or 'postalCode' in col or 'name.nativeName' in col or 'name.tsn' in col or 'idd' in col or 'altSpellings' in col or 'gini' in col]
df.drop(columns=df_cols_to_drop, inplace=True)

In [84]:
df.columns.value_counts()

tld                    1
demonyms.fra.m         1
startOfWeek            1
name.common            1
name.official          1
demonyms.eng.f         1
demonyms.eng.m         1
demonyms.fra.f         1
maps.googleMaps        1
timezones              1
maps.openStreetMaps    1
car.signs              1
car.side               1
flags.png              1
flags.svg              1
flags.alt              1
continents             1
fifa                   1
cca2                   1
region                 1
ccn3                   1
cioc                   1
independent            1
status                 1
unMember               1
capital                1
subregion              1
population             1
latlng                 1
landlocked             1
borders                1
area                   1
cca3                   1
flag                   1
capitalInfo.latlng     1
dtype: int64